# YOLOv5モデルのファインチューニング

このNotebookでは、COCO事前学習済みYOLOv5nモデルをKaggle OCRデータセットでファインチューニングします。

**高速化のため、ローカルストレージ（/content/tmp/）を使用します。**

## 処理の流れ
1. Google Driveのマウント
2. Gitリポジトリのクローン/更新
3. 環境変数の設定
4. データセットをローカルストレージにコピー（高速化）
5. データセットの確認
6. 事前学習済みモデルの準備
7. 学習設定
8. 学習の実行
9. 学習結果の確認
10. 最良モデルをGoogle Driveに保存


## GPUランタイムとコンピューティングユニットについて

**⚠️ 重要: このNotebookは最初からGPUランタイムで実行してください。**

- CPUランタイムでデータコピー → GPUランタイムに変更すると、`/content/`配下のデータが消えます
- GPUランタイムに変更すると、ランタイムがリセットされ、ローカルストレージのデータが失われます
- データコピー中もGPUランタイムが有効なため、コンピューティングユニットが消費されます

### GPU/TPUの特徴とコストパフォーマンス比較

| デバイス | アーキテクチャ | メモリ | 主な特徴 | 5時間あたりCU消費（推定） | コスパ評価 | YOLOv5適性 |
|---------|------------|--------|---------|----------------------|----------|-----------|
| **L4** | Ada Lovelace | 24GB | T4の後継、性能向上（FP16: 242 TFLOPS）、低消費電力（72W） | **~10-12 CU** | ⭐⭐⭐⭐⭐ 最良 | ✅ 推奨（最適） |
| **T4** | Turing | 16GB | 低コスト、広く利用可能（FP16: 65 TFLOPS）、低消費電力（70W） | **7.44 CU** | ⭐⭐⭐⭐ 優秀 | ✅ 推奨（コスト重視） |
| **A100** | Ampere | 80GB | 高性能（FP16: 624 TFLOPS）、大規模モデル向け | **~30-40 CU** | ⭐⭐⭐ 良好 | ✅ 高性能が必要な場合 |
| **H100** | Hopper | 80GB | 最新・最高性能（FP16: 1,513 TFLOPS）、Colabでは通常利用不可 | **~50-75 CU** | ⭐⭐ 低い | ⚠️ 過剰性能・利用困難 |
| **TPU v6e-1** | TPU v6e | 32GB | 高性能（BF16: 918 TFLOPS）、TensorFlow/JAX向け | - | - | ❌ **非推奨**（PyTorch非対応） |
| **TPU v5e-1** | TPU v5e | 16GB | コスト効率向上、TensorFlow/JAX向け | - | - | ❌ **非推奨**（PyTorch非対応） |

**推奨順位**:
1. **L4 GPU**: 性能とコストのバランスが最良。T4の後継で推奨。
2. **T4 GPU**: コストが最も低く、確実に動作。時間に余裕がある場合に最適。
3. **A100 GPU**: 高性能が必要で予算がある場合に検討。

**注意事項**:
- **TPUは非推奨**: YOLOv5はPyTorchベース（ultralytics）のため、TPUでは動作しない可能性が高い
- ColabではGPUの割り当てがランダムな場合があります。希望するGPUが割り当てられない可能性があります
- CU消費量はColab Pro（月額100ユニット付与）を基準としています
- L4、A100、H100のCU消費量は推定値です（T4の実測値に基づく相対比較）



## 使用方法について

このNotebookは`create_yolo_dataset.ipynb`の後に実行してください。

- **前提**: `create_yolo_dataset.ipynb`で画像とラベルの1対1対応が完成していること
- **入力**: `processed/{split}/images/` と `processed/{split}/labels/`
- **出力**: 学習済みモデル（`.pt`ファイル）をGoogle Driveに保存
- **結果**: ファインチューニング済みYOLOv5nモデル

### 注意事項
- **ローカルストレージを使用**: Google Driveへの直接アクセスは遅いため、一時的にローカルストレージ（/content/tmp/）を使用します
- **GPU使用**: **最初からGPUランタイムで実行してください**（ランタイム → ランタイムのタイプを変更 → GPU）。CPUランタイムからGPUランタイムに変更すると、ローカルストレージのデータが消えます
- **段階的fine-tuning**: このNotebookは段階2（Kaggleデータ）用です。段階3（自前データ）は別途実行してください


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content
!git clone -b feature/onnx_recognizer https://github.com/masararutaru/last_assignment_progzissen.git
# %cd last_assignment_progzissen
# !git pull  # 毎回実行

# 環境変数の設定
import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"
os.environ["OUT_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"

print("環境変数設定完了")
print(f"DATA_ROOT: {os.environ.get('DATA_ROOT')}")
print(f"OUT_ROOT: {os.environ.get('OUT_ROOT')}")


## データセットをローカルストレージにコピー（高速化・train/val分割）

Google Driveから直接読み込むと遅いため、ローカルストレージにコピーします。
**初回のみ実行が必要です（数分かかります）。**

**重要**: このセルでは、データを**train（80%）とval（20%）に自動分割**してコピーします。
- 画像とラベルのペアを維持しながらランダムに分割
- 再現性のため、ランダムシードを固定（`random_seed=42`）
- YOLOv5が自動的にtrain/valを認識できる構造で保存


In [ ]:
# データセットをローカルストレージにコピー（train/val分割付き）
import shutil
import random
import os
from pathlib import Path
from tqdm import tqdm

# 環境変数からパスを取得
out_root = os.environ.get("OUT_ROOT")
if not out_root:
    raise ValueError(
        "❌ エラー: OUT_ROOT環境変数が設定されていません。\n"
        "   セル3で環境変数を設定してください。"
    )

split = 'train'  # ソースデータはtrainフォルダ

# Google Driveのデータセットパス
drive_data_dir = Path(out_root) / "processed" / split
drive_images_dir = drive_data_dir / "images"
drive_labels_dir = drive_data_dir / "labels"

# ローカルストレージのパス（train/valに分割）
local_dataset_dir = Path("/content/tmp/dataset")
local_train_images_dir = local_dataset_dir / "train" / "images"
local_train_labels_dir = local_dataset_dir / "train" / "labels"
local_val_images_dir = local_dataset_dir / "val" / "images"
local_val_labels_dir = local_dataset_dir / "val" / "labels"

# 分割比率
train_ratio = 0.8  # 80% train, 20% val
random_seed = 42  # 再現性のためのシード

print("=" * 80)
print("データセットのローカルコピー（train/val分割）")
print("=" * 80)
print(f"ソース（Google Drive）: {drive_data_dir}")
print(f"  画像: {drive_images_dir}")
print(f"  ラベル: {drive_labels_dir}")
print(f"出力先（ローカル）: {local_dataset_dir}")
print(f"分割比率: {train_ratio*100:.0f}% train, {(1-train_ratio)*100:.0f}% val")
print(f"ランダムシード: {random_seed}")
print()

# 既にコピー済みか確認
if local_train_images_dir.exists() and local_train_labels_dir.exists() and \
   local_val_images_dir.exists() and local_val_labels_dir.exists():
    train_image_count = len(list(local_train_images_dir.glob('*.*')))
    train_label_count = len(list(local_train_labels_dir.glob('*.txt')))
    val_image_count = len(list(local_val_images_dir.glob('*.*')))
    val_label_count = len(list(local_val_labels_dir.glob('*.txt')))
    print(f"✓ ローカルデータセットは既に存在します")
    print(f"  Train: 画像 {train_image_count} 件, ラベル {train_label_count} 件")
    print(f"  Val: 画像 {val_image_count} 件, ラベル {val_label_count} 件")
    print(f"\n再コピーする場合は、以下のコマンドを実行してください:")
    print(f"  !rm -rf {local_dataset_dir}")
else:
    # ソースの存在確認
    if not drive_images_dir.exists():
        raise FileNotFoundError(
            f"❌ エラー: 画像ディレクトリが見つかりません: {drive_images_dir}\n"
            "   先に create_yolo_dataset.ipynb を実行してください。"
        )
    
    if not drive_labels_dir.exists():
        raise FileNotFoundError(
            f"❌ エラー: ラベルディレクトリが見つかりません: {drive_labels_dir}\n"
            "   先に preprocess_kaggle_data.ipynb を実行してください。"
        )
    
    # ファイルリストを取得（画像とラベルのペアを確認）
    print("ファイルリストを取得中...")
    image_files = list(drive_images_dir.glob('*.*'))
    label_files = list(drive_labels_dir.glob('*.txt'))
    
    # UUIDでペアリング
    image_uuids = {f.stem: f for f in image_files}
    label_uuids = {f.stem: f for f in label_files}
    matched_uuids = sorted(set(image_uuids.keys()) & set(label_uuids.keys()))
    
    print(f"\n見つかったファイル:")
    print(f"  画像ファイル: {len(image_files)} 件")
    print(f"  ラベルファイル: {len(label_files)} 件")
    print(f"  対応しているペア数: {len(matched_uuids)} 件")
    
    if len(matched_uuids) == 0:
        raise ValueError("❌ エラー: 対応している画像とラベルのペアが見つかりません")
    
    # ランダムにシャッフル（シード固定で再現性を確保）
    random.seed(random_seed)
    shuffled_uuids = matched_uuids.copy()
    random.shuffle(shuffled_uuids)
    
    # train/valに分割
    split_idx = int(len(shuffled_uuids) * train_ratio)
    train_uuids = shuffled_uuids[:split_idx]
    val_uuids = shuffled_uuids[split_idx:]
    
    print(f"\n分割結果:")
    print(f"  Train: {len(train_uuids)} 件 ({len(train_uuids)/len(shuffled_uuids)*100:.1f}%)")
    print(f"  Val: {len(val_uuids)} 件 ({len(val_uuids)/len(shuffled_uuids)*100:.1f}%)")
    print()
    
    # ディレクトリ作成
    local_train_images_dir.mkdir(parents=True, exist_ok=True)
    local_train_labels_dir.mkdir(parents=True, exist_ok=True)
    local_val_images_dir.mkdir(parents=True, exist_ok=True)
    local_val_labels_dir.mkdir(parents=True, exist_ok=True)
    
    # Trainデータをコピー
    print("=" * 80)
    print("Trainデータをコピー中...")
    print("=" * 80)
    copied_train_images = 0
    copied_train_labels = 0
    
    for uuid in tqdm(train_uuids, desc="Trainコピー", unit="ペア"):
        # 画像をコピー
        src_img = image_uuids[uuid]
        dst_img = local_train_images_dir / src_img.name
        if not dst_img.exists():
            shutil.copy2(src_img, dst_img)
            copied_train_images += 1
        
        # ラベルをコピー
        src_label = label_uuids[uuid]
        dst_label = local_train_labels_dir / src_label.name
        if not dst_label.exists():
            shutil.copy2(src_label, dst_label)
            copied_train_labels += 1
    
    print(f"Trainコピー完了: 画像 {copied_train_images} 件, ラベル {copied_train_labels} 件")
    
    # Valデータをコピー
    print("\n" + "=" * 80)
    print("Valデータをコピー中...")
    print("=" * 80)
    copied_val_images = 0
    copied_val_labels = 0
    
    for uuid in tqdm(val_uuids, desc="Valコピー", unit="ペア"):
        # 画像をコピー
        src_img = image_uuids[uuid]
        dst_img = local_val_images_dir / src_img.name
        if not dst_img.exists():
            shutil.copy2(src_img, dst_img)
            copied_val_images += 1
        
        # ラベルをコピー
        src_label = label_uuids[uuid]
        dst_label = local_val_labels_dir / src_label.name
        if not dst_label.exists():
            shutil.copy2(src_label, dst_label)
            copied_val_labels += 1
    
    print(f"Valコピー完了: 画像 {copied_val_images} 件, ラベル {copied_val_labels} 件")
    
    print("\n" + "=" * 80)
    print("✓ コピー完了")
    print("=" * 80)
    print(f"Train: {len(train_uuids)} ペア")
    print(f"Val: {len(val_uuids)} ペア")
    print(f"\nローカルパス: {local_dataset_dir}")


In [ ]:
# データセットの確認
import sys
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# パスを追加（リポジトリのpython/dataディレクトリ）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

# ローカルデータセットのパス（train/valに分割済み）
local_dataset_dir = Path("/content/tmp/dataset")
local_train_images_dir = local_dataset_dir / "train" / "images"
local_train_labels_dir = local_dataset_dir / "train" / "labels"
local_val_images_dir = local_dataset_dir / "val" / "images"
local_val_labels_dir = local_dataset_dir / "val" / "labels"

print("=" * 80)
print("データセット確認")
print("=" * 80)

# Trainデータの確認
print("\nTrainデータを確認中...")
train_image_files = list(local_train_images_dir.glob('*.*'))
train_label_files = list(local_train_labels_dir.glob('*.txt'))

print(f"Train - 画像ファイル数: {len(train_image_files)} 件")
print(f"Train - ラベルファイル数: {len(train_label_files)} 件")

# Valデータの確認
print("\nValデータを確認中...")
val_image_files = list(local_val_images_dir.glob('*.*'))
val_label_files = list(local_val_labels_dir.glob('*.txt'))

print(f"Val - 画像ファイル数: {len(val_image_files)} 件")
print(f"Val - ラベルファイル数: {len(val_label_files)} 件")

# 1対1対応の確認（Train）
train_image_uuids = {f.stem for f in train_image_files}
train_label_uuids = {f.stem for f in train_label_files}
train_matched = train_image_uuids & train_label_uuids

print(f"\nTrain - 1対1対応:")
print(f"  画像ファイル: {len(train_image_uuids)} 件")
print(f"  ラベルファイル: {len(train_label_uuids)} 件")
print(f"  対応しているペア数: {len(train_matched)} 件")

# 1対1対応の確認（Val）
val_image_uuids = {f.stem for f in val_image_files}
val_label_uuids = {f.stem for f in val_label_files}
val_matched = val_image_uuids & val_label_uuids

print(f"\nVal - 1対1対応:")
print(f"  画像ファイル: {len(val_image_uuids)} 件")
print(f"  ラベルファイル: {len(val_label_uuids)} 件")
print(f"  対応しているペア数: {len(val_matched)} 件")

if len(train_matched) != len(train_image_uuids) or len(train_matched) != len(train_label_uuids):
    print(f"\n⚠️  警告: Trainデータで一部のファイルが対応していません")
if len(val_matched) != len(val_image_uuids) or len(val_matched) != len(val_label_uuids):
    print(f"\n⚠️  警告: Valデータで一部のファイルが対応していません")

# クラス別統計（Train）
print("\n" + "=" * 80)
print("Trainデータ - クラス別統計を集計中...")
print("=" * 80)

train_class_stats = defaultdict(int)
train_total_instances = 0

for label_file in tqdm(train_label_files, desc="Trainラベル解析", unit="ファイル"):
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if 0 <= class_id < len(CLASSES):
                    train_class_stats[CLASSES[class_id]] += 1
                    train_total_instances += 1

print(f"\nTrain - クラス別インスタンス数:")
print("-" * 60)
for cls in CLASSES:
    count = train_class_stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")
print("-" * 60)
print(f"Train - 総インスタンス数: {train_total_instances}")

# クラス別統計（Val）
print("\n" + "=" * 80)
print("Valデータ - クラス別統計を集計中...")
print("=" * 80)

val_class_stats = defaultdict(int)
val_total_instances = 0

for label_file in tqdm(val_label_files, desc="Valラベル解析", unit="ファイル"):
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if 0 <= class_id < len(CLASSES):
                    val_class_stats[CLASSES[class_id]] += 1
                    val_total_instances += 1

print(f"\nVal - クラス別インスタンス数:")
print("-" * 60)
for cls in CLASSES:
    count = val_class_stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")
print("-" * 60)
print(f"Val - 総インスタンス数: {val_total_instances}")

if train_total_instances == 0:
    print("\n⚠️  警告: Trainデータにインスタンスが見つかりませんでした")
if val_total_instances == 0:
    print("\n⚠️  警告: Valデータにインスタンスが見つかりませんでした")


In [ ]:
# ultralyticsのインストール（必要に応じて）
!pip install ultralytics -q

from ultralytics import YOLO
from pathlib import Path
import sys

# パスを追加（CLASSESをインポートするため）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

print("=" * 80)
print("事前学習済みモデルの準備")
print("=" * 80)

# YOLOv5nモデルを読み込み（初回実行時に自動ダウンロード）
print("\nYOLOv5nモデルを読み込み中...")
print("（初回実行時は自動ダウンロードされます。約4.5MB、数秒かかります）")

model = YOLO('yolov5n.pt')

print("\n✓ モデル読み込み完了")
print(f"  モデル名: YOLOv5n")
print(f"  クラス数: {len(CLASSES)} クラス")
print(f"  クラス: {', '.join(CLASSES)}")


## 学習設定

学習パラメータを設定します。段階2（Kaggleデータ）用の設定です。


In [ ]:
# YOLOデータセット設定ファイル（data.yaml）の作成
!pip install pyyaml -q

import yaml
from pathlib import Path
import sys

# パスを追加（CLASSESをインポートするため）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

data_yaml_path = Path("/content/tmp/dataset/data.yaml")

data_config = {
    'path': str(Path("/content/tmp/dataset").absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(CLASSES),
    'names': CLASSES
}

with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

# ファイルの存在確認
if not data_yaml_path.exists():
    raise FileNotFoundError(
        f"❌ エラー: データセット設定ファイルの作成に失敗しました: {data_yaml_path}"
    )

print("=" * 80)
print("YOLOデータセット設定ファイル作成")
print("=" * 80)
print(f"設定ファイル: {data_yaml_path}")
print(f"\n内容:")
print(yaml.dump(data_config, default_flow_style=False))

# data_pathをyamlファイルに変更
data_path = str(data_yaml_path)
print(f"\n✓ データセットパスを更新: {data_path}")


In [ ]:
# 学習設定
from pathlib import Path
import sys

# データセット設定ファイルの確認
if 'data_path' not in globals():
    raise ValueError(
        "❌ エラー: データセット設定ファイルが作成されていません。\n"
        "   先にセル12（YOLOデータセット設定ファイル作成）を実行してください。"
    )

# パスを追加（CLASSESをインポートするため）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

# split変数の定義（セル6で定義されているが、念のため再定義）
split = 'train'  # ソースデータはtrainフォルダ

# データセットパス（セル12でdata.yamlに設定済み）
# data_pathはセル12で設定されているので、ここでは再設定しない

# 学習パラメータ
epochs = 50  # エポック数
batch_size = 16  # バッチサイズ（GPUメモリに応じて調整）
# GPUメモリ容量別の推奨バッチサイズ目安:
#   - 8GB  (例: RTX 3060, RTX 3070): batch_size = 8 程度
#   - 12GB (例: RTX 3060 12GB): batch_size = 12 程度
#   - 16GB (例: P100, T4): batch_size = 16 程度（現在の設定）
#   - 24GB (例: RTX 3090, RTX 4090, A100): batch_size = 24 程度
img_size = 640  # 画像サイズ
device = 0  # GPU使用（0: GPU, cpu: CPU）

# 出力先（ローカルストレージ）
project_path = "/content/tmp/runs"
run_name = f"yolov5n_kaggle_{split}"

print("=" * 80)
print("学習設定")
print("=" * 80)
print(f"データセット: {data_path}")
print(f"エポック数: {epochs}")
print(f"バッチサイズ: {batch_size}")
print(f"画像サイズ: {img_size}")
print(f"デバイス: {'GPU' if device == 0 else 'CPU'}")
print(f"出力先: {project_path}/{run_name}")
print(f"\nクラス数: {len(CLASSES)} クラス")
print(f"クラス: {', '.join(CLASSES)}")


## 学習の実行

**注意**: このセルは長時間かかります（数時間）。GPUランタイムを使用してください。


In [ ]:
# 学習の実行
print("=" * 80)
print("学習開始")
print("=" * 80)
print(f"開始時刻: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# 学習実行
results = model.train(
    data=data_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    device=device,
    project=project_path,
    name=run_name,
    save=True,
    save_period=10,  # 10エポックごとに保存
    val=True,  # 検証データで評価
    plots=True,  # 学習曲線をプロット
    verbose=True,  # 詳細ログ
)

print("\n" + "=" * 80)
print("学習完了")
print("=" * 80)
print(f"終了時刻: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## 学習結果の確認


In [ ]:
# 学習結果の確認
from pathlib import Path
import json

# 学習設定の確認
if 'project_path' not in globals() or 'run_name' not in globals():
    raise ValueError(
        "❌ エラー: 学習設定が完了していません。\n"
        "   先にセル13（学習設定）を実行してください。"
    )

results_dir = Path(project_path) / run_name
weights_dir = results_dir / "weights"

print("=" * 80)
print("学習結果")
print("=" * 80)
print(f"結果ディレクトリ: {results_dir}")

# 重みファイルの確認
if weights_dir.exists():
    weight_files = list(weights_dir.glob('*.pt'))
    print(f"\n重みファイル数: {len(weight_files)} 件")
    
    for wf in sorted(weight_files):
        size_mb = wf.stat().st_size / (1024 * 1024)
        print(f"  {wf.name}: {size_mb:.2f} MB")
    
    # 最良モデル
    best_model = weights_dir / "best.pt"
    if best_model.exists():
        print(f"\n✓ 最良モデル: {best_model.name}")
        print(f"  サイズ: {best_model.stat().st_size / (1024 * 1024):.2f} MB")
    
    # 最終モデル
    last_model = weights_dir / "last.pt"
    if last_model.exists():
        print(f"✓ 最終モデル: {last_model.name}")
        print(f"  サイズ: {last_model.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print("\n⚠️  重みファイルが見つかりません")

# 学習曲線の確認
results_csv = results_dir / "results.csv"
if results_csv.exists():
    print(f"\n学習曲線データ: {results_csv}")
    print("  （results.csvをダウンロードしてExcel等で確認できます）")

# プロット画像の確認
plot_files = list(results_dir.glob("*.png"))
if len(plot_files) > 0:
    print(f"\nプロット画像数: {len(plot_files)} 件")
    for pf in plot_files:
        print(f"  {pf.name}")


## 学習結果をGoogle Driveに保存

学習済みモデル、学習曲線、プロット画像を全てGoogle Driveに保存して、次回以降も使用できるようにします。

**保存内容**:
- 最良モデル（best.pt）
- 最終モデル（last.pt、存在する場合）
- 学習曲線データ（results.csv）
- プロット画像（*.png）

**フォルダ構成**:
学習設定（データセット、エポック数、バッチサイズ）を含むフォルダ名で保存します。
例: `yolov5n_kaggle_train_epochs50_batch16_20240101_120000`


In [ ]:
# 学習結果をGoogle Driveに保存
import shutil
import os
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# 環境変数からパスを取得
out_root = os.environ.get("OUT_ROOT")
if not out_root:
    raise ValueError(
        "❌ エラー: OUT_ROOT環境変数が設定されていません。\n"
        "   セル4で環境変数を設定してください。"
    )

# 学習設定の確認
if 'project_path' not in globals() or 'run_name' not in globals() or \
   'split' not in globals() or 'epochs' not in globals() or 'batch_size' not in globals():
    raise ValueError(
        "❌ エラー: 学習設定が完了していません。\n"
        "   先にセル13（学習設定）を実行してください。"
    )

# ローカルの結果ディレクトリ
local_results_dir = Path(project_path) / run_name
local_weights_dir = local_results_dir / "weights"

if not local_results_dir.exists():
    raise FileNotFoundError(
        f"❌ エラー: 学習結果ディレクトリが見つかりません: {local_results_dir}\n"
        "   先に学習を実行してください。"
    )

# フォルダ名を作成（学習設定を含む）
# 形式: yolov5n_kaggle_{split}_epochs{epochs}_batch{batch_size}_{timestamp}
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"yolov5n_kaggle_{split}_epochs{epochs}_batch{batch_size}_{timestamp}"

# Google Driveの保存先（weightsディレクトリ配下に学習結果フォルダを作成）
drive_weights_dir = Path(out_root) / "weights"
drive_results_dir = drive_weights_dir / folder_name
drive_results_dir.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("学習結果をGoogle Driveに保存")
print("=" * 80)
print(f"ソース: {local_results_dir}")
print(f"保存先: {drive_results_dir}")
print(f"フォルダ名: {folder_name}")
print()

# コピーするファイルのリスト
files_to_copy = []

# 1. 最良モデル（best.pt）
local_best_model = local_weights_dir / "best.pt"
if local_best_model.exists():
    files_to_copy.append(("最良モデル", local_best_model, drive_results_dir / "best.pt"))
else:
    print("⚠️  最良モデル（best.pt）が見つかりません")

# 2. 最終モデル（last.pt）
local_last_model = local_weights_dir / "last.pt"
if local_last_model.exists():
    files_to_copy.append(("最終モデル", local_last_model, drive_results_dir / "last.pt"))

# 3. 学習曲線データ（results.csv）
local_results_csv = local_results_dir / "results.csv"
if local_results_csv.exists():
    files_to_copy.append(("学習曲線データ", local_results_csv, drive_results_dir / "results.csv"))
else:
    print("⚠️  学習曲線データ（results.csv）が見つかりません")

# 4. プロット画像（*.png）
plot_files = list(local_results_dir.glob("*.png"))
for plot_file in plot_files:
    files_to_copy.append((f"プロット画像: {plot_file.name}", plot_file, drive_results_dir / plot_file.name))

if len(plot_files) == 0:
    print("⚠️  プロット画像（*.png）が見つかりません")

print(f"\n保存するファイル数: {len(files_to_copy)} 件")
print()

# ファイルをコピー
copied_count = 0
total_size_mb = 0

for desc, src_file, dst_file in files_to_copy:
    if not src_file.exists():
        print(f"⚠️  スキップ: {desc}（ファイルが見つかりません）")
        continue
    
    file_size = src_file.stat().st_size
    size_mb = file_size / (1024 * 1024)
    total_size_mb += size_mb
    
    print(f"コピー中: {desc}")
    print(f"  サイズ: {size_mb:.2f} MB")
    
    # 大きなファイルの場合は進捗表示付きでコピー
    if file_size > 10 * 1024 * 1024:  # 10MB以上の場合
        chunk_size = 1024 * 1024  # 1MB
        with open(src_file, 'rb') as src, open(dst_file, 'wb') as dst:
            with tqdm(total=file_size, unit='B', unit_scale=True, desc="  ", leave=False) as pbar:
                while True:
                    chunk = src.read(chunk_size)
                    if not chunk:
                        break
                    dst.write(chunk)
                    pbar.update(len(chunk))
    else:
        # 小さいファイルは通常のコピー
        shutil.copy2(src_file, dst_file)
    
    copied_count += 1
    print(f"  ✓ 完了\n")

print("=" * 80)
print("✓ 保存完了")
print("=" * 80)
print(f"保存したファイル数: {copied_count} 件")
print(f"合計サイズ: {total_size_mb:.2f} MB")
print(f"\n保存先: {drive_results_dir}")
print(f"\n次回以降は以下のパスから読み込めます:")
print(f"  最良モデル: {drive_results_dir / 'best.pt'}")
if local_results_csv.exists():
    print(f"  学習曲線: {drive_results_dir / 'results.csv'}")
if len(plot_files) > 0:
    print(f"  プロット画像: {drive_results_dir}/*.png")
